In [3]:
from openai import OpenAI
import base64
from PIL import Image
import json
import time

# Base64 编码格式
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# 读取 JSON 文件
json_path = r'D:\Number_Sense\others\Area\Area.json'

client = OpenAI(
    api_key='sk-1a78cb15a70e41eea2e41bd81420cd6c',
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 读取 JSON 数据
with open(json_path, 'r') as json_file:
    data = json.load(json_file)

correct_answer = 0
for item in data:
    text = item['text']
    image = item['image']
    option = item['option'] + ", please output the answer directly."
    answer = item['answer']
    base64_image = encode_image(image)

    max_retries = 5
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model="qwen-vl-max",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "image_url",
                                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                            },
                            {"type": "text", "text": text + option},
                        ],
                    }
                ],
            )
            output = completion.choices[0].message.content
            print(output)
            print(answer)
            if answer == output[0]:
                correct_answer += 1
            break  # 如果请求成功，则跳出重试循环

        except Exception as e:
                wait_time = 2 ** attempt  # 指数回退
                print(f"Rate limit exceeded. Waiting for {wait_time} seconds...")
                time.sleep(wait_time)  # 等待

    time.sleep(1)  # 在每个请求之间增加固定的延迟
print(f"the correct_answer number is {correct_answer}")

FileNotFoundError: [Errno 2] No such file or directory: 'Number_Sense/Chart_Game_data/Picture/Area/image\\0_yellow.png'